Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training/train-within-notebook/train-within-notebook.png)

# Train and deploy a model
_**Create and deploy a model directly from a notebook**_

---
---

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
    1. Viewing run results
    1. Simple parameter sweep
    1. Viewing experiment results
    1. Select the best model
1. [Deploy](#Deploy)
    1. Register the model
    1. Create a scoring file
    1. Create the environment yml file
    1. Create the Docker Image
    1. Deploy the Docker image as web service on Azure Container Instance
    1. Test the Web Service
    1. Clean up
1. [Next Steps](#nextsteps)

---

## Introduction
Azure Machine Learning provides capabilities to control all aspects of model training and deployment directly from a notebook using the AML Python SDK.  In this notebook we will
* connect to our AML Workspace
* create an experiment that contains multiple runs with tracked metrics
* choose the best model created across all runs
* deploy that model as a service

In the end we will have a model deployed as a web service which we can call from an HTTP endpoint

---

## Setup
Ensure you have created an Azure Machine Learning (AML) workspace with a running compute instance.Then, open this notebook from within the AML Studio and attach it to the compute instance.

For this notebook we need the Azure ML SDK and access to our workspace.  The following cell imports the SDK, checks the version, and accesses our already configured AzureML workspace.

In [ ]:
# Import all required libraries
# If some of these libraries fail, run pip list and pip install to make sure all required libraries are installed on this environment

import azureml.core
import joblib
import json
import numpy as np
import os
import pandas as pd
from azureml.core import Experiment, Workspace, Dataset
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.fsspec import AzureMachineLearningFileSystem
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Check core SDK version number
print("This notebook was created using version 1.0.2 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")


ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

---

## Data
We will use the diabetes dataset for this experiment, a well-known small dataset that comes with scikit-learn. The dataset consists of ten baseline variables: age, sex, body mass index, average blood pressure, and six blood serum measurements that were obtained for each of n = 442 diabetes patients, as well as a quantitative measure of disease progression one year after baseline, as described in [scikit-learn.org](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) website. This cell demonstrates how to load this dataset directly from sciket-learn library (load_diabetes) 


In [ ]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y = True)

print ("Data contains", len(X), '442 diabetes patients')

For the purpose of this demo, We have created a CSV file from the scikit-learn diabetes dataset that we will use instead. We need to create an AML [Datastore](https://learn.microsoft.com/en-us/azure/machine-learning/concept-data#datastore) (or use the default one), and then create an AML [Dataset](https://learn.microsoft.com/en-us/azure/machine-learning/concept-data#data-asset) and upload the CSV file into it. You can create the AML Datastore and Dataset intuitively in the AML Studio or through the Azure CLI/Python SDK. You can find more details [here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-datastore?tabs=cli-identity-based-access%2Ccli-adls-identity-based-access%2Ccli-azfiles-account-key%2Ccli-adlsgen1-identity-based-access). You can find the CSV file in the [GitHub Repo](https://github.com/MustAl-Du/AzureMLDemo/blob/main/diabetesDataSet.csv)

In [ ]:
# Variables for the AML Datastore
datastore_name = 'workspaceblobstore'
datasource_relativepath = 'UI/2023-03-03_214901_UTC/diabetesDataSet.csv'

uri = 'azureml://subscriptions/'+ws.subscription_id+'/resourcegroups/'+ws.resource_group+'/workspaces/'+ws.name+'/datastores/'+datastore_name+'/paths/'+datasource_relativepath

print(uri)

In [ ]:
# instantiate file system using datastore URI
fs = AzureMachineLearningFileSystem(uri)

# use an open context
with fs.open('/'+datastore_name+'/'+datasource_relativepath) as f:
        dataset = pd.read_csv(f)

print("Loading Diabetes Data from the CSV file...")
X, y = dataset[['age', 'gender', 'bmi', 'bp', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']].values, dataset['y'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data = {
    "train":{"X": X_train, "y": y_train},        
    "test":{"X": X_test, "y": y_test}
}

print ("Data contains", len(data['train']['X']), "training samples and",len(data['test']['y']), "test samples")


---
## Train

Let's use scikit-learn to train a simple Ridge regression model.  We use AML to record interesting information about the model in an Experiment.  An Experiment contains a series of trials called Runs.  During this trial we use AML in the following way:
* We access an experiment from our AML workspace by name, which will be created if it doesn't exist
* We use `start_logging` to create a new run in this experiment
* We use `run.log()` to record a parameter, alpha, and an accuracy measure - the Mean Squared Error (MSE) to the run.  We will be able to review and compare these measures in the Azure Portal at a later time.
* We store the resulting model in the **working** directory, which is automatically captured by AML when the run is complete.
* We use `run.complete()` to indicate that the run is over and results can be captured and finalized

In [ ]:
# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="train-within-notebook-for-powerbi")

# Create a run object in the experiment
run =  experiment.start_logging()
# Log the algorithm parameter alpha to the run; where alpha is between 0 and 1
run.log('alpha', 0.03)

# Create, fit, and test the scikit-learn Ridge regression model
regression_model = Ridge(alpha=0.03)
regression_model.fit(data['train']['X'], data['train']['y'])
preds = regression_model.predict(data['test']['X'])

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
run.log('mse', mean_squared_error(data['test']['y'], preds))

# Save the model to the working directory 
model_file_name = 'diabetesmodel.pkl'

joblib.dump(value = regression_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()

### Viewing run results
Azure Machine Learning stores all the details about the run in the Azure cloud.  Let's access those details by retrieving a link to the run using the default run output.  Clicking on the resulting link will take you to an interactive page presenting all run information.

In [ ]:
run

### Simple parameter sweep
Now let's take the same concept from above and modify the **alpha** parameter.  For each value of alpha we will create a run that will store metrics and the resulting model.  In the end we can use the captured run history to determine which model was the best for us to deploy. 

Note that by using `with experiment.start_logging() as run` AML will automatically call `run.complete()` at the end of each loop.

This example also uses the **tqdm** library to provide a thermometer feedback

In [ ]:
# list of numbers from 0 to 1.0 with a 0.10 interval
alphas = np.arange(0.0, 1.0, 0.10)

# try a bunch of alpha values in a Linear Regression (Ridge) model
for alpha in tqdm(alphas):
    # create a bunch of runs, each train a model with a different alpha value
    with experiment.start_logging() as run:
        # Use Ridge algorithm to build a regression model
        regression_model = Ridge(alpha=alpha)
        regression_model.fit(X=data["train"]["X"], y=data["train"]["y"])
        preds = regression_model.predict(X=data["test"]["X"])
        mse = mean_squared_error(y_true=data["test"]["y"], y_pred=preds)

        # log alpha, mean_squared_error and feature names in run history
        run.log(name="alpha", value=alpha)
        run.log(name="mse", value=mse)

        # Save the model to the outputs directory for capture
        joblib.dump(value=regression_model, filename='diabetesmodel.pkl')


### Viewing experiment results
Similar to viewing the run, we can also view the entire experiment.  The experiment report view in the Azure portal lets us view all the runs in a table, and also allows us to customize charts.  This way, we can see how the alpha parameter impacts the quality of the model

In [ ]:
# now let's take a look at the experiment in Azure portal.
experiment

### Select the best model 
Now that we've created many runs with different parameters, we need to determine which model is the best for deployment.  For this, we will iterate over the set of runs.  From each run we will take the *run id* using the `id` property, and examine the metrics by calling `run.get_metrics()`.  

Since each run may be different, we do need to check if the run has the metric that we are looking for, in this case, **mse**.  To find the best run, we create a dictionary mapping the run id's to the metrics.

Finally, we use the `tag` method to mark the best run to make it easier to find later. 

In [ ]:
runs = {}
run_metrics = {}

# Create dictionaries containing the runs and the metrics for all runs containing the 'mse' metric
for r in tqdm(experiment.get_runs()):
    metrics = r.get_metrics()
    if 'mse' in metrics.keys():
        runs[r.id] = r
        run_metrics[r.id] = metrics

# Find the run with the best (lowest) mean squared error and display the id and metrics
best_run_id = min(run_metrics, key = lambda k: run_metrics[k]['mse'])
best_run = runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', run_metrics[best_run_id])

# Tag the best run for identification later
best_run.tag("Best Run")

---
## Deploy
Now that we have trained a set of models and identified the run containing the best model, we want to deploy the model for real time inference.  The process of deploying a model involves
* registering a model in your workspace
* creating a scoring file containing init and run methods
* creating an environment dependency file describing packages necessary for your scoring file
* deploying the model and packages as a web service

### Register a model
We have already identified which run contains the "best model" by our evaluation criteria.  Each run has a file structure associated with it that contains various files collected during the run.  Since a run can have many outputs we need to tell AML which file from those outputs represents the model that we want to use for our deployment.  We can use the `run.get_file_names()` method to list the files associated with the run, and then use the `run.register_model()` method to place the model in the workspace's model registry.

When using `run.register_model()` we supply a `model_name` that is meaningful for our scenario and the `model_path` of the model relative to the run.  In this case, the model path is what is returned from `run.get_file_names()`

In [ ]:
# View the files in the run
for f in best_run.get_file_names():
    print(f)
    
# Register the model with the workspace
model = Model.register(model_path = "diabetesmodel.pkl",
                       model_name = "diabetesmodel.pkl",
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace =ws)

Once a model is registered, it is accessible from the list of models on the AML workspace.  If you register models with the same name multiple times, AML keeps a version history of those models for you.  The `Model.list()` lists all models in a workspace, and can be filtered by name, tags, or model properties.   

In [ ]:
# Find all models called "best_model" and display their version numbers
models = Model.list(ws, name='diabetesmodel.pkl')
for m in models:
    print(m.name, m.version)

### Create a scoring file

Since your model file can essentially be anything you want it to be, you need to supply a scoring script that can load your model and then apply the model to new data. This script is your 'scoring file'. This scoring file is a python program containing, at a minimum, two methods init() and run(). The init() method is called once when your deployment is started so you can load your model and any other required objects. This method uses the get_model_path function to locate the registered model inside the docker container. The run() method is called interactively when the web service is called with one or more data samples to predict.

Important: The schema decorators for pandas and numpy are required to implement the automatic swagger schema generation for input and output variables

After a successful run of the this script, the score.py file be created in the working folder


In [ ]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model



from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def init():
    global model
    model_path = Model.get_model_path('diabetesmodel.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

input_sample = pd.DataFrame(data=[{
            "input1_age": 59,
            "input2_gender": 2,
            "input3_bmi": 32,
            "input4_bp": 101,
            "input5_tc": 157,
            "input6_ldl": 93,
            "input7_hdl": 38,
            "input8_tch": 4,
            "input9_ltg": 5,
            "input10_glu": 87
            }])
output_sample = np.array([0])

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

def run(data):
    try:
        print("input_data....")
        print(data.columns)
        print(type(data))
        result = model.predict(data)
        print("result.....")
        print(result)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Create the environment yml file

This step will create the yml file (myenv.yml) in the working folder for this deployment



In [ ]:
env = Environment('deploytocloudenv')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy','pandas', 'scikit-learn'],pip_packages=['azureml-defaults','inference-schema[numpy-support]'])
inference_config = InferenceConfig(entry_script="score.py", environment=env)

with open ("myenv.yml","w") as f:
   f.write(env.python.conda_dependencies.serialize_to_string())


### Deploy the Docker image as a web service on Azure Container Instance

Note that the service creation can take few minutes.

In [ ]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "diabetes", 'type': "regression"}, 
                                               description = 'aci web service with the diabetes regression model',
                                               location = 'East US')

In [ ]:
# Create the webservice using all of the precreated configurations and our best model
aciWebservice = Model.deploy(workspace=ws,
                       name='aci-webservice-diabetesmodel',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aciconfig,
                       overwrite=True)

# Wait for the service deployment to complete while displaying log output
aciWebservice.wait_for_deployment(show_output=True)
print(aciWebservice.state)
print(aciWebservice.get_logs)

In [ ]:
# Check out the swagger URI for the deployed ACI WebService
aciWebservice.swagger_uri

### Test web service

Call the web service with some dummy input data to get a prediction.

In [ ]:
test_sample = json.dumps({"data": [{
        "input1_age": 59,
        "input2_sex": 2,
        "input3_bmi": 32,
        "input4_bp": 101,
        "input5_s1": 157,
        "input6_s2": 93,
        "input7_s3": 38,
        "input8_s4": 4,
        "input9_s5": 5,
        "input10_s10": 87,}]})

test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aciWebservice.run(input_data=test_sample)
print(prediction)
print(round(prediction[0]))

### Clean up

Delete the ACI instance to stop the compute and any associated billing.

In [ ]:
%%time
aciWebservice.delete()

<a id='nextsteps'></a>
## Next Steps

In this example, you created a models inside the notebook using local data, stored them inside an AML experiment, found the best one and deployed it as a live service!  From here you can continue to use Azure Machine Learning in this regard to run your own experiments and deploy your own models, or you can expand into further capabilities of AML!

Now, you are ready to use Power BI Desktop to infer this model leveraging the native integration between Power BI and AML. Refer to the instruction in [Power BI Integration with Azure ML Model](https://github.com/MustAl-Du/AzureMLDemo/blob/main/Power%20BI%20Integration%20with%20Azure%20ML%20Model.docx).